In [1]:
import pandas as pd

unmatch_data = pd.read_csv('unmatch_shops.csv')

In [7]:
# data = data.groupby('shop_index').sum()
# unmatch_data.info()
# unmatch_data = unmatch_data.drop(columns=['Unnamed: 0','decorated_indicator'])
unmatch_data = unmatch_data[unmatch_data['performance_date']==8]
unmatch_data.head()
unmatch_data.info()

X = unmatch_data.drop("masked_order",1)
y = unmatch_data["masked_order"]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7980 entries, 2 to 111717
Data columns (total 19 columns):
manage_shop_indicator            7980 non-null int64
officialstore_indicator          7980 non-null int64
preferred_shop_indicator         7980 non-null int64
crossborder_indicator            7980 non-null int64
shop_category                    7980 non-null int64
new_seller_flag                  7980 non-null int64
seller_centre_login_L30D         7980 non-null int64
shop_sku_number                  7980 non-null int64
shop_follower_number             7980 non-null int64
shop_L180D_order                 7980 non-null int64
weighted_shop_rating             7980 non-null float64
performance_date                 7980 non-null int64
masked_item_impression           7980 non-null float64
masked_order                     7980 non-null int64
masked_shop_page_view            7980 non-null int64
masked_shop_click_from_search    7980 non-null float64
masked_campaign_tab_click        7980

In [8]:
#get unmatched shop features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

#apply SelectKBest class to extract top 5 best features
bestfeatures = SelectKBest(score_func=f_regression, k=5)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(5,'Score'))  #print 5 best features

                       Specs         Score
12    masked_item_impression  31752.359471
9           shop_L180D_order  11731.706780
16    masked_other_tab_click  11259.677052
8       shop_follower_number   5831.141102
2   preferred_shop_indicator     66.551904


/Users/eva/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/Users/eva/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:307: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/Users/eva/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/eva/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/eva/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [17]:
#get features of whitelist shops
w_df = pd.read_csv('whitelist_filled_latest.csv')
w_df = w_df.drop(['Unnamed: 0','decorated_indicator'],1)

w_df = w_df[w_df['performance_date'] == 8]
# w_df = w_df.groupby('shop_index').mean()


w_df = w_df.drop(['masked_campaign_tab_click'],1)
w_x = w_df.drop("masked_order",1)
w_y = w_df["masked_order"]

#apply SelectKBest class to extract top 5 best features
bestfeatures = SelectKBest(score_func=f_regression, k=5)
fit = bestfeatures.fit(w_x,w_y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(w_x.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(5,'Score'))  #print 5 best features


                            Specs  Score
15         masked_other_tab_click 809.04
14  masked_shop_click_from_search 422.63
9                shop_L180D_order 331.45
8            shop_follower_number 267.03
12         masked_item_impression  86.95


/Users/eva/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/Users/eva/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:307: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/Users/eva/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/eva/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/eva/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [19]:
w_df.head()
w_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 11 to 7493
Data columns (total 18 columns):
manage_shop_indicator            536 non-null float64
officialstore_indicator          536 non-null float64
preferred_shop_indicator         536 non-null float64
crossborder_indicator            536 non-null float64
shop_category                    536 non-null float64
new_seller_flag                  536 non-null float64
seller_centre_login_L30D         536 non-null float64
shop_sku_number                  536 non-null float64
shop_follower_number             536 non-null float64
shop_L180D_order                 536 non-null float64
weighted_shop_rating             536 non-null float64
performance_date                 536 non-null float64
masked_item_impression           536 non-null float64
masked_order                     536 non-null float64
masked_shop_page_view            536 non-null float64
masked_shop_click_from_search    536 non-null float64
masked_other_tab_click       

In [20]:
#calculate the coefficient 
feature = featureScores.nlargest(5,'Score')
feature['Score'] = feature['Score']/(feature['Score'].sum())
feature

,Specs,Score
15,masked_other_tab_click,0.42
14,masked_shop_click_from_search,0.22
9,shop_L180D_order,0.17
8,shop_follower_number,0.14
12,masked_item_impression,0.05


In [21]:
#calculate the score of whitelist shops
features = list(feature['Specs'])
scores = list(feature['Score'])
equation = ''
for i in range(len(features)):
    equation  += str(scores[i]) +'*'+str(features[i])+'+'
equation = 'Score='+ equation[:len(equation)-1]
equation
wf_process = w_df[features]
wf_process = wf_process.eval(equation)
pd.set_option('display.float_format',lambda x : '%.2f' % x)
# wf_process.to_csv('whitelist_with_score.csv')
wf_process 




,masked_other_tab_click,masked_shop_click_from_search,shop_L180D_order,shop_follower_number,masked_item_impression,Score
11,34004.00,2209.00,156119.00,82005.00,11099000000000.00,503391847429.90
16,135.00,1.00,24078.00,3132.00,280093696.00,12708221.36
40,366.00,1.00,47179.00,13398.00,1084582489.00,49201089.11
45,27.89,9.53,405.00,1203.00,47961.00,2426.72
62,542.00,1.00,10012.00,12903.00,376243609.00,17068172.50
71,993.00,4.00,11611.00,2211.00,2963604721.00,134416132.76
96,106.05,134.37,405.00,1053.00,15563025.00,706147.37
102,14.00,9.00,431.00,4233.00,1144900.00,52598.62
113,47.00,127.79,14562.00,2052.00,53260804.00,2418479.17
138,267.00,578.68,30396.00,26613.00,4144526884.00,187982967.98


In [45]:
import numpy as np
wf_process['Score'].describe()
wf_process.sort_values("Score",inplace=True)

threshold = np.percentile(wf_process['Score'],50)
threshold  

28792173.051724736

In [47]:
#get score for all unmatched shops
# unmatch_data.set_index(["shop_index"], inplace=True)
unmatch_data
un_data_process = unmatch_data[features]
un_data_process = un_data_process.eval(equation)
un_data_process
# data_process.to_csv('unmatch_with_score.csv')

un_data_process.sort_values("Score",inplace=True)
un_data_process['Score'].describe()
un_data_process.head()

,masked_other_tab_click,masked_shop_click_from_search,shop_L180D_order,shop_follower_number,masked_item_impression,Score
19126,16.89,1.74,405,972,0.00,212.92
36154,16.89,1.74,405,972,0.00,212.92
2559,16.89,1.74,405,972,0.00,212.92
9731,16.89,1.74,405,972,0.00,212.92
5504,16.89,1.74,405,972,0.00,212.92


In [48]:
select_shop = un_data_process[un_data_process['Score']>threshold]

# select_shop = select_shop.groupby('shop_index').mean()
select_shop

,masked_other_tab_click,masked_shop_click_from_search,shop_L180D_order,shop_follower_number,masked_item_impression,Score
31086,190.00,98.05,23701,3696,637562500.00,28921169.03
104493,190.00,98.05,18267,3252,637865536.00,28933911.80
29938,36.00,4.00,4760,1794,649995025.00,29481417.05
28548,267.00,25.00,14328,14718,651015225.00,29531244.44
90058,113.00,66.11,405,2496,670136769.00,30394330.79
72029,58.00,65.16,405,2160,676676169.00,30690853.08
45534,1213.00,36.00,2537,3708,680844649.00,30880978.45
30396,25.00,11.58,5618,1581,680896836.00,30883075.06
27840,861.00,961.00,5007,4869,686125636.00,31121140.16
105448,388.00,81.00,4279,3420,698492041.00,31681293.42


In [25]:
select_shop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6577 entries, 24 to 111717
Data columns (total 6 columns):
masked_other_tab_click           6577 non-null float64
masked_shop_click_from_search    6577 non-null float64
shop_L180D_order                 6577 non-null int64
shop_follower_number             6577 non-null int64
masked_item_impression           6577 non-null float64
Score                            6577 non-null float64
dtypes: float64(4), int64(2)
memory usage: 359.7 KB


In [36]:
#score for unmatched data
un_data_process['Score'].describe()

count           7980.00
mean        86693592.98
std       3070118587.51
min              212.92
25%              771.26
50%            26874.18
75%           295438.46
max     229953281667.28
Name: Score, dtype: float64

In [37]:
#perform equation on matched shops
match_df = pd.read_csv('match_with_label.csv')
match_df = match_df.drop(['Unnamed: 0','decorated_indicator'],1)
match_df = match_df.drop(['masked_campaign_tab_click'],1)
match_df



match_process = match_df[features]
match_process = match_process.eval(equation)
pd.set_option('display.float_format',lambda x : '%.2f' % x)
# wf_process.to_csv('whitelist_with_score.csv')
match_process 
#记得加进去datecolumn


,masked_other_tab_click,masked_shop_click_from_search,shop_L180D_order,shop_follower_number,masked_item_impression,Score
0,21.53,1.42,405,975,25.00,216.36
1,14.00,1.00,405,975,36.00,213.59
2,21.53,1.47,405,975,121.00,220.73
3,25.58,1.00,405,975,9.00,217.25
4,19.79,1.00,405,975,225.00,224.61
5,23.84,1.00,405,975,1.00,216.16
6,25.00,1.00,405,975,4.00,216.78
7,18.05,1.47,405,975,64.00,216.68
8,18.05,1.47,405,975,49.00,215.99
9,22.11,1.16,405,975,100.00,219.95


In [26]:
#score of matched shops
match_process['Score'].describe()

count           252532.00
mean         496375567.69
std        88229769931.21
min                113.37
25%                121.09
50%                198.02
75%               4818.65
max     33719730177769.17
Name: Score, dtype: float64

In [27]:
#score of whitelist shop
wf_process['Score'].describe()

count             7504.00
mean        9291614023.74
std       172574113786.31
min                119.28
25%             515177.86
50%           13457752.54
75%          170504258.71
max     10606572483257.32
Name: Score, dtype: float64

In [28]:
#score for unmatched data
un_data_process['Score'].describe()

count         111720.00
mean        41370823.33
std       2030310290.62
min              114.22
25%              524.16
50%            20026.26
75%           214157.10
max     496252535290.79
Name: Score, dtype: float64

In [49]:
select_shop.info()
final = pd.con

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 31086 to 87234
Data columns (total 6 columns):
masked_other_tab_click           258 non-null float64
masked_shop_click_from_search    258 non-null float64
shop_L180D_order                 258 non-null int64
shop_follower_number             258 non-null int64
masked_item_impression           258 non-null float64
Score                            258 non-null float64
dtypes: float64(4), int64(2)
memory usage: 14.1 KB


In [ ]:
select_shop.to_csv('selected_unmatch.csv')